# Raw Data Processing

## DBLP paper ciation data loading

In [143]:
file_path = '/Users/zongchang/Desktop/实验数据集/DBLP-Paper'

In [144]:
with open(file_path + '/DBLPOnlyCitationOct19.txt', 'r') as f:
    paper_list = []
    paper = {}
    paper_raw = []
    for line in f.readlines():
        if line == '\n':
            ref_index = []
            for item in paper_raw:
                if item.startswith('#*'):
                    paper['title'] = item.replace('#*', '')
                elif item.startswith('#@'):
                    paper['author'] = item.replace('#@', '')
                elif item.startswith('#t'):
                    paper['year'] = item.replace('#t', '')
                elif item.startswith('#c'):
                    paper['venue'] = item.replace('#c', '')
                elif item.startswith('#index'):
                    paper['index'] = item.replace('#index', '')
                elif item.startswith('#%'):
                    ref_index.append(item.replace('#%', ''))
                elif item.startswith('#!'):
                    paper['abstract'] = item.replace('#!', '')
                else:
                    continue
            paper['ref_index'] = ref_index
            paper_list.append(paper)
            paper = {}
            paper_raw = []
        else:
            paper_raw.append(line.strip())

In [149]:
# filter paper with low quality: absence of properties, earlier papers, small number of references
paper_list_filter = []
for paper in paper_list:
    if ('abstract' not in paper) or ('title' not in paper) or ('author' not in paper) or (int(paper['year']) < 2000) or len(paper['ref_index'])<10:
        continue
    else:
        paper_list_filter.append(paper)

In [150]:
len(paper_list_filter)

66501

## DBLP paper citation count (indicator_input.pkl)

In [151]:
# index yearly citation dict for papers at year 2000
index_cite_count = {}
for paper in paper_list_filter:
    if paper['year'] == '2000':
        index_cite_count[paper['index']] = {}

# count citation
for paper in paper_list_filter:
    for ref in paper['ref_index']:
        if ref in index_cite_count:
            if paper['year'] not in index_cite_count[ref]:
                index_cite_count[ref][paper['year']] = 1
            else:
                index_cite_count[ref][paper['year']] += 1

        

In [152]:
# keep papers have citations
index_cite_count_nonzero = {}
for k,v in index_cite_count.items():
    if v:
        index_cite_count_nonzero[k] = v

In [153]:
years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']
citation_count_list = [] # index, year, citation_count
for k,v in index_cite_count_nonzero.items():
    for year in years:
        if year not in v:
            tmp = [k, year, 0]
        else:
            tmp = [k, year, v[year]]
        citation_count_list.append(tmp)

In [194]:
citation_count_list

[['5652', '2000', 0],
 ['5652', '2001', 0],
 ['5652', '2002', 0],
 ['5652', '2003', 1],
 ['5652', '2004', 4],
 ['5652', '2005', 0],
 ['5652', '2006', 1],
 ['5652', '2007', 2],
 ['5652', '2008', 1],
 ['5652', '2009', 4],
 ['5652', '2010', 0],
 ['5693', '2000', 0],
 ['5693', '2001', 0],
 ['5693', '2002', 2],
 ['5693', '2003', 0],
 ['5693', '2004', 1],
 ['5693', '2005', 0],
 ['5693', '2006', 1],
 ['5693', '2007', 2],
 ['5693', '2008', 0],
 ['5693', '2009', 1],
 ['5693', '2010', 0],
 ['6516', '2000', 1],
 ['6516', '2001', 0],
 ['6516', '2002', 0],
 ['6516', '2003', 0],
 ['6516', '2004', 0],
 ['6516', '2005', 0],
 ['6516', '2006', 0],
 ['6516', '2007', 1],
 ['6516', '2008', 0],
 ['6516', '2009', 0],
 ['6516', '2010', 0],
 ['9722', '2000', 0],
 ['9722', '2001', 0],
 ['9722', '2002', 2],
 ['9722', '2003', 0],
 ['9722', '2004', 0],
 ['9722', '2005', 0],
 ['9722', '2006', 1],
 ['9722', '2007', 1],
 ['9722', '2008', 0],
 ['9722', '2009', 1],
 ['9722', '2010', 0],
 ['13378', '2000', 0],
 ['13378'

In [155]:
import pickle
with open('./data/dblp_paper/paper_1749_citation.pkl', 'wb') as f:
    pickle.dump(citation_count_list, f)

In [156]:
idx_2_paper_index = {}
idx = 0
for paper_index in list(index_cite_count_nonzero.keys()):
    idx_2_paper_index[idx] = paper_index
    idx += 1

In [157]:
indicator_citation = []
for idx in range(len(list(idx_2_paper_index.keys()))):
    tmp = []
    for indicator in citation_count_list:
        if indicator[0] == idx_2_paper_index[idx]:
            tmp.append([[indicator[2]], indicator[1]])
    
    indicator_citation.append(tmp)

In [158]:
indicator_citation

[[[[0], '2000'],
  [[0], '2001'],
  [[0], '2002'],
  [[1], '2003'],
  [[4], '2004'],
  [[0], '2005'],
  [[1], '2006'],
  [[2], '2007'],
  [[1], '2008'],
  [[4], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[0], '2001'],
  [[2], '2002'],
  [[0], '2003'],
  [[1], '2004'],
  [[0], '2005'],
  [[1], '2006'],
  [[2], '2007'],
  [[0], '2008'],
  [[1], '2009'],
  [[0], '2010']],
 [[[1], '2000'],
  [[0], '2001'],
  [[0], '2002'],
  [[0], '2003'],
  [[0], '2004'],
  [[0], '2005'],
  [[0], '2006'],
  [[1], '2007'],
  [[0], '2008'],
  [[0], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[0], '2001'],
  [[2], '2002'],
  [[0], '2003'],
  [[0], '2004'],
  [[0], '2005'],
  [[1], '2006'],
  [[1], '2007'],
  [[0], '2008'],
  [[1], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[0], '2001'],
  [[0], '2002'],
  [[1], '2003'],
  [[2], '2004'],
  [[4], '2005'],
  [[0], '2006'],
  [[1], '2007'],
  [[0], '2008'],
  [[0], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[0], '2001'],
  [[0], '2002'],
  [[0], '

In [159]:
import pickle
with open('./data/dblp_paper/indicator_year_input.pkl', 'wb') as f:
    pickle.dump(indicator_citation, f)

## DBLP paper graph (graph_input.pkl)

In [163]:
# extract author and citation relations from high quality papers
paper_author = {}
paper_citation = {}
author_list = []
paper_list = []
years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']

for item in paper_list_filter:
    author_list.extend(item['author'].split(','))
    paper_list.append(item['index'])
    paper_author[item['index']] = {'author': item['author'].split(','), 'year': item['year']}
    paper_citation[item['index']] = {'paper': item['ref_index'], 'year': item['year']}
    

In [166]:
author_list = list(set(author_list))
paper_list = list(set(paper_list))

In [164]:
len(list(paper_author.keys()))

66501

In [165]:
len(list(paper_citation.keys()))

66501

In [167]:
# continue add paper index
exist_paper_index = list(idx_2_paper_index.values())
idx = max(list(idx_2_paper_index.keys())) + 1
for paper_index in paper_list:
    if paper_index in exist_paper_index:
        continue
    else:
        idx_2_paper_index[idx] = paper_index
        idx += 1

In [169]:
max(list(idx_2_paper_index.keys()))

66500

In [170]:
idx_2_author_name = {}
idx = 70000
for author_name in author_list:
    idx_2_author_name[idx] = author_name
    idx += 1

In [171]:
author_name_2_idx = {}
for k,v in idx_2_author_name.items():
    author_name_2_idx[v] = k

paper_index_2_idx = {}
for k,v in idx_2_paper_index.items():
    paper_index_2_idx[v] = k

In [172]:
idx_2_author_name

{70000: 'Kouichi Sakurai',
 70001: 'Sathish Gopalakrishnan',
 70002: 'Hemant Tagare',
 70003: 'Jonathan de Halleux',
 70004: 'Karina Hauser',
 70005: 'Zhigang Peng',
 70006: 'Edwin K. P. Chong',
 70007: 'Michael Dittenbach',
 70008: 'Yuan-Tse Yu',
 70009: 'Paul Thompson',
 70010: 'Thierry Mora',
 70011: 'Natalie Lee-San Pang',
 70012: 'Traci A. Carte',
 70013: 'Z. M. Ma',
 70014: 'Junyu Dong',
 70015: 'Emmanuel Juin',
 70016: 'Deng Cai',
 70017: "Andrea D'Ambrogio",
 70018: 'Janne Riihijärvi',
 70019: 'James A. Storer',
 70020: 'Jagoba Arias',
 70021: 'Riko Jacob',
 70022: 'Peter R. Wilson',
 70023: 'Ravi Janardan',
 70024: 'Darrell C. Anderson',
 70025: 'Yingwei Cui',
 70026: 'Alan Halverson',
 70027: 'Matthew Suderman',
 70028: 'Reetuparna Das',
 70029: 'Christopher Schwarz',
 70030: 'Ferdi A. Smit',
 70031: 'Su-Ping Chen',
 70032: 'Daniela Giorgetti',
 70033: 'Zhongnan Zhang',
 70034: 'Vishal J. Mehta',
 70035: 'Rami Beidas',
 70036: 'Robert C. Goldstein',
 70037: 'Farshad Fotouhi',

In [173]:
len(list(idx_2_author_name.keys()))

74395

In [174]:
idx_2_paper_index

{0: '5652',
 1: '5693',
 2: '6516',
 3: '9722',
 4: '13378',
 5: '17568',
 6: '17575',
 7: '17759',
 8: '19853',
 9: '20105',
 10: '20181',
 11: '20383',
 12: '20494',
 13: '20534',
 14: '20634',
 15: '20952',
 16: '21087',
 17: '21116',
 18: '23291',
 19: '23292',
 20: '23293',
 21: '39252',
 22: '39260',
 23: '39305',
 24: '39518',
 25: '39789',
 26: '39861',
 27: '41420',
 28: '41428',
 29: '41440',
 30: '41447',
 31: '41473',
 32: '41523',
 33: '41546',
 34: '41588',
 35: '41600',
 36: '41604',
 37: '41671',
 38: '41672',
 39: '44834',
 40: '50066',
 41: '53584',
 42: '53600',
 43: '53610',
 44: '53625',
 45: '53665',
 46: '53701',
 47: '53713',
 48: '53732',
 49: '53755',
 50: '53758',
 51: '53761',
 52: '53791',
 53: '53831',
 54: '53835',
 55: '60916',
 56: '60939',
 57: '60955',
 58: '61021',
 59: '61048',
 60: '61053',
 61: '61078',
 62: '61125',
 63: '62742',
 64: '71315',
 65: '71927',
 66: '77045',
 67: '77100',
 68: '77108',
 69: '77195',
 70: '77277',
 71: '77314',
 72: '

In [175]:
paper_author_idx = {}
for k,v in paper_author.items():
    paper_author_idx[paper_index_2_idx[k]] = {'author': [author_name_2_idx[item] for item in v['author']], 'year': v['year']}

In [176]:
paper_author_idx

{23333: {'author': [82210, 126930], 'year': '2003'},
 54982: {'author': [88346, 123957], 'year': '2003'},
 51116: {'author': [103107, 123957, 123010], 'year': '2003'},
 26669: {'author': [91157], 'year': '2003'},
 2882: {'author': [135240, 112147, 98020], 'year': '2003'},
 26654: {'author': [123348, 82552, 123995], 'year': '2003'},
 23472: {'author': [77339, 127058, 139042], 'year': '2003'},
 3309: {'author': [119626, 118034, 104362, 80829], 'year': '2003'},
 31232: {'author': [87459, 73641, 122377, 101879, 82983], 'year': '2003'},
 42141: {'author': [82210, 126930], 'year': '2002'},
 50805: {'author': [106374, 94159], 'year': '2005'},
 50136: {'author': [91947], 'year': '2005'},
 11522: {'author': [84390, 80487, 140799, 122593], 'year': '2003'},
 35816: {'author': [118425, 125463, 114507], 'year': '2003'},
 62122: {'author': [116840, 132181], 'year': '2003'},
 55456: {'author': [85925, 78041], 'year': '2003'},
 24617: {'author': [113519], 'year': '2003'},
 9508: {'author': [113519, 81

In [139]:
paper_citation = {}
for item in paper_list_filter:
    if item['index'] in index_cite_count_nonzero:
        paper_citation[item['index']] = item['ref_index']

In [179]:
paper_citation_idx = {}
for k,v in paper_citation.items():
    tmp = []
    for item in v['paper']:
        if item in paper_index_2_idx:
            tmp.append(item)
    paper_citation_idx[paper_index_2_idx[k]] = {'paper': [paper_index_2_idx[item] for item in tmp], 'year': v['year']}

In [180]:
paper_citation_idx

{23333: {'paper': [829, 1500, 17973], 'year': '2003'},
 54982: {'paper': [1000, 829, 17973], 'year': '2003'},
 51116: {'paper': [1300], 'year': '2003'},
 26669: {'paper': [], 'year': '2003'},
 2882: {'paper': [], 'year': '2003'},
 26654: {'paper': [52990, 64141], 'year': '2003'},
 23472: {'paper': [], 'year': '2003'},
 3309: {'paper': [1357, 758, 18613, 1036], 'year': '2003'},
 31232: {'paper': [19949, 708], 'year': '2003'},
 42141: {'paper': [1500], 'year': '2002'},
 50805: {'paper': [37465, 650], 'year': '2005'},
 50136: {'paper': [27679, 42152, 642, 1360, 1363], 'year': '2005'},
 11522: {'paper': [1360, 52990], 'year': '2003'},
 35816: {'paper': [1363], 'year': '2003'},
 62122: {'paper': [582], 'year': '2003'},
 55456: {'paper': [], 'year': '2003'},
 24617: {'paper': [], 'year': '2003'},
 9508: {'paper': [1363], 'year': '2003'},
 55401: {'paper': [], 'year': '2003'},
 46181: {'paper': [1363], 'year': '2003'},
 55864: {'paper': [60897], 'year': '2003'},
 44204: {'paper': [26329], 'ye

In [188]:
import networkx as nx

years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']
graph_list = []
for year in years:
    G_author = nx.Graph()
    G_citation = nx.Graph()
    G_author.add_nodes_from(list(idx_2_paper_index.keys()))
    G_author.add_nodes_from(list(idx_2_author_name.keys()))
    G_citation.add_nodes_from(list(idx_2_paper_index.keys()))
    G_citation.add_nodes_from(list(idx_2_author_name.keys()))

    for k,v in paper_author_idx.items():
        if v['year'] == year:
            for author in v['author']:
                G_author.add_edge(k, author)

    for k,v in paper_citation_idx.items():
        if v['year'] == year:
            for paper in v['paper']:
                G_citation.add_edge(k, paper)
    graph_list.append([G_author, G_citation, idx_2_paper_index, idx_2_author_name])


In [208]:

for i in range(len(graph_list)):
    print(years[i] + '  Author edges added this year: ' + str(nx.number_of_edges(graph_list[i][0])) + '  Citation edges added this year: ' + str(nx.number_of_edges(graph_list[i][1])))


2000  Author edges added this year: 5591  Citation edges added this year: 700
2001  Author edges added this year: 7289  Citation edges added this year: 2409
2002  Author edges added this year: 11369  Citation edges added this year: 5863
2003  Author edges added this year: 14973  Citation edges added this year: 11031
2004  Author edges added this year: 15720  Citation edges added this year: 15673
2005  Author edges added this year: 18381  Citation edges added this year: 22307
2006  Author edges added this year: 21897  Citation edges added this year: 29890
2007  Author edges added this year: 31517  Citation edges added this year: 46847
2008  Author edges added this year: 31171  Citation edges added this year: 50386
2009  Author edges added this year: 32528  Citation edges added this year: 56900
2010  Author edges added this year: 8247  Citation edges added this year: 15060


In [207]:
for i in range(len(graph_list)):
    print(years[i] + '  Snapshot nodes this year: ' + str(nx.number_of_nodes(graph_list[i][0])))

2000  Snapshot nodes this year: 140896
2001  Snapshot nodes this year: 140896
2002  Snapshot nodes this year: 140896
2003  Snapshot nodes this year: 140896
2004  Snapshot nodes this year: 140896
2005  Snapshot nodes this year: 140896
2006  Snapshot nodes this year: 140896
2007  Snapshot nodes this year: 140896
2008  Snapshot nodes this year: 140896
2009  Snapshot nodes this year: 140896
2010  Snapshot nodes this year: 140896


In [198]:
with open('./data/dblp_paper/graph_input.pkl', 'wb') as f:
    pickle.dump(graph_list, f)

In [199]:
paper_num = {}
for year in years:
    tmp = []
    for k,v in paper_citation_idx.items():
        if v['year'] == year:
            tmp.append(k)
    paper_num[year] = len(list(set(tmp)))

In [205]:
for k,v in paper_num.items():
    print(k + '  Papers published this year: ' + str(v))

2000  Papers published this year: 2102
2001  Papers published this year: 2703
2002  Papers published this year: 4155
2003  Papers published this year: 5333
2004  Papers published this year: 5390
2005  Papers published this year: 6208
2006  Papers published this year: 7299
2007  Papers published this year: 10377
2008  Papers published this year: 10081
2009  Papers published this year: 10290
2010  Papers published this year: 2563


## DBLP paper text content (doc_input.pkl)